In [47]:
import torch
from torch.utils.data import DataLoader, Dataset
import nltk
import string
from nltk.corpus import stopwords
import torch.nn as nn
import yaml
from tqdm import tqdm
nltk.download('stopwords')
import matplotlib.pyplot as plt
import numpy as np
import re
from gensim.models import Word2Vec
import gensim.downloader
from torchtext.vocab import build_vocab_from_iterator, Vocab

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jazzblazzer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('iNLP-A2/ANLP-2/test.csv')

# Display the data
print(df['Description'][0])


Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.


In [49]:
START_TOKEN = "<s>"
END_TOKEN = "</s>"
UNKNOWN_TOKEN = "<unk>"
PAD_TOKEN = "<pad>"

In [50]:
class LSTMDataset:
    def __init__(self, dataset, tokenizer, max_length):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        tokenized_sentence = self.tokenizer(self.dataset[idx])
        return [START_TOKEN]+tokenized_sentence,tokenized_sentence+[END_TOKEN]

        
    
    
lstm_data = LSTMDataset(df['Description'],nltk.word_tokenize, 100)
print(lstm_data[0])

(['<s>', 'Unions', 'representing', 'workers', 'at', 'Turner', 'Newall', 'say', 'they', 'are', "'disappointed", "'", 'after', 'talks', 'with', 'stricken', 'parent', 'firm', 'Federal', 'Mogul', '.'], ['Unions', 'representing', 'workers', 'at', 'Turner', 'Newall', 'say', 'they', 'are', "'disappointed", "'", 'after', 'talks', 'with', 'stricken', 'parent', 'firm', 'Federal', 'Mogul', '.', '</s>'])


## Embeddings

Getting pre trained embeddings.

In [55]:
# wor2vec_model = gensim.downloader.load('word2vec-google-news-300')#word2vec-google-news-300
vocab = build_vocab_from_iterator([nltk.word_tokenize(i) for i in df['Description']], specials=[PAD_TOKEN, UNKNOWN_TOKEN, START_TOKEN, END_TOKEN])
vocab.set_default_index(vocab[UNKNOWN_TOKEN])
print([nltk.word_tokenize(df['Description'][2054])])
print(vocab['Khartoum'])


[['AFP', '-', 'Sudan', 'has', 'condemned', 'as', 'quot', ';', 'unfair', 'quot', ';', 'a', 'new', 'UN', 'resolution', 'calling', 'on', 'Khartoum', 'to', 'restore', 'security', 'to', 'the', 'crisis-wracked', 'Darfur', 'region', 'or', 'face', 'possible', 'sanctions', ',', 'but', 'said', 'it', 'would', 'abide', 'by', 'the', 'UN', "'s", 'demands', '.']]
5021


In [8]:

# embedding_matrix = torch.zeros(len(vocab), wor2vec_model.vector_size)
# for i, word in enumerate(vocab.get_itos()):
#     if word in wor2vec_model:
#         embedding_matrix[i] = torch.tensor(wor2vec_model[word])
#     else:
#         embedding_matrix[i] = torch.tensor(wor2vec_model["unk"])
#change this to random later

In [33]:
print(vocab['hello'])

21822


In [56]:
from torch.nn.utils.rnn import pad_sequence
def collate(batch):
    inputs = []
    targets = []
    for i in range(len(batch)):

        inputs.append(torch.tensor([vocab[word] if word in vocab else vocab[UNKNOWN_TOKEN] for word in batch[i][0]]))
        targets.append(torch.tensor([vocab[word] if word in vocab else vocab[UNKNOWN_TOKEN] for word in batch[i][1]]))
    return pad_sequence(inputs, batch_first=True, padding_value=vocab[PAD_TOKEN]),pad_sequence(targets, batch_first=True, padding_value=vocab[PAD_TOKEN])

In [57]:
lstm_data_loader = DataLoader(lstm_data, batch_size=32, shuffle=True, collate_fn=collate)

In [58]:
print(len(lstm_data_loader))

238


In [59]:
for i in range(1):
    for inputs, targets in lstm_data_loader:
        print(inputs)
        print(targets)
        break
print(vocab.get_itos()[2])
print(vocab['Khartoum'])

tensor([[    2,  7495,  5873,  ...,     0,     0,     0],
        [    2,   102,   110,  ...,     0,     0,     0],
        [    2,   319,    14,  ...,     0,     0,     0],
        ...,
        [    2,   224,  4193,  ...,     0,     0,     0],
        [    2,    52,    14,  ...,     0,     0,     0],
        [    2, 17588,   243,  ...,    34,    62,    12]])
tensor([[ 7495,  5873,  1634,  ...,     0,     0,     0],
        [  102,   110,    21,  ...,     0,     0,     0],
        [  319,    14,    42,  ...,     0,     0,     0],
        ...,
        [  224,  4193, 20338,  ...,     0,     0,     0],
        [   52,    14,    20,  ...,     0,     0,     0],
        [17588,   243,   728,  ...,    62,    12,     3]])
<s>
5021


In [60]:


if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
print(device)



cpu


ELMO architecture.
2 layer bi lstm as (flstm1,rlstm1,flstm2,rlstm2)
Returns these values, which are then concatenated and made of same size by padding (or duplicating)
Leanable parameters have been done using the learnable parameters of pytorch, where 3 values were learned.(one for each bilstm layer and 1 for pretrained embedding)
Used required_grad flag to freeze the elmo model and trained downstream.
for func, i used an nn.Linear()
and random with np.random.
All accuracy values are shown in report.

Learning - 0.71
Fixed(random) - 0.64
Function - 0.76

We see for random the lowest, but not very bad compared to others, which is to be expected.
For Function we have more complex linear transforms to learn hence the increase from normal learnable parameters.


In [62]:
class LM(nn.Module):
    def __init__(self,vocab,batch_size, hidden_size,output_size):
        super().__init__()
        self.embedding = nn.Embedding(len(vocab), hidden_size)
        self.flstm1 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.flstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.rlstm1 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.rlstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.output = nn.Linear(2*hidden_size, output_size)

        
    def forward(self, x):
        x = self.embedding(x)
        o_f1,_ = self.flstm1(x)
        o_f2,_ = self.flstm2(o_f1)
        o_r1,_ = self.rlstm1(torch.flip(x, [1]))
        o_r2,_ = self.rlstm2(o_r1)
        return x,o_f1,o_f2,o_r1,o_r2
    
    def elmo(self, x,batch_size):
        x,of1,of2,or1,or2 = self.forward(x)
        or2 = torch.flip(or2, dims=(1,))
        cat = torch.cat((of2,or2),axis=2)
        return self.output(cat)


LanguageModel = LM(vocab=vocab,
                batch_size=32,
               hidden_size=512,
               output_size=len(vocab))

In [ ]:
print(len())

In [64]:
num_epochs = 10

# Lists to store training and validation losses
train_loss = []
val_loss = []
LanguageModel.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(LanguageModel.parameters(), lr=0.1)
c = 10
for epoch in range(num_epochs):
    LanguageModel.train()
    epoch_train_loss = 0.0
    for batch in lstm_data_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = LanguageModel.elmo(inputs,inputs.shape[0])
        outputs = outputs.view(-1, outputs.shape[-1])
        labels = labels.view(-1)
        print(labels.shape,outputs.shape)
        break
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
        print(loss.item())
        c-=1
        if c < 0:
            break
    epoch_train_loss /= len(lstm_data_loader)
    train_loss.append(epoch_train_loss)
    LanguageModel.eval()
    epoch_val_loss = 0.0
    torch.save(LanguageModel, f"elmo.pt")


torch.Size([1856]) torch.Size([1856, 25974])
torch.Size([2592]) torch.Size([2592, 25974])
torch.Size([1504]) torch.Size([1504, 25974])
torch.Size([2720]) torch.Size([2720, 25974])
torch.Size([1824]) torch.Size([1824, 25974])
torch.Size([2592]) torch.Size([2592, 25974])
torch.Size([2368]) torch.Size([2368, 25974])
torch.Size([4800]) torch.Size([4800, 25974])
torch.Size([1760]) torch.Size([1760, 25974])
torch.Size([1632]) torch.Size([1632, 25974])


In [69]:
LanguageModel = torch.load("elmo.pt", map_location=device)
LanguageModel.eval()
vocab_list = vocab.get_itos()
for batch in lstm_data_loader:
    inputs, labels = batch
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = LanguageModel.elmo(inputs,inputs.shape[0])
    print(vocab_list[torch.argmax(outputs[0][0])],vocab_list[torch.argmax(outputs[0][1])])
    print(vocab_list[labels[0][0]],vocab_list[labels[0][1]])
    print(vocab_list[torch.argmax(outputs[1][0])],vocab_list[torch.argmax(outputs[1][1])],vocab_list[torch.argmax(outputs[1][2])],vocab_list[torch.argmax(outputs[1][3])],vocab_list[torch.argmax(outputs[1][4])])
    print(vocab_list[labels[1][0]],vocab_list[labels[1][1]],vocab_list[labels[1][2]],vocab_list[labels[1][3]],vocab_list[labels[1][4]])
    print(vocab_list[torch.argmax(outputs[2][0])],vocab_list[torch.argmax(outputs[2][1])],vocab_list[torch.argmax(outputs[2][2])],vocab_list[torch.argmax(outputs[2][3])],vocab_list[torch.argmax(outputs[2][4])])
    break   


Fannie Fannie
The Big
A on from from that
A Fort Worth brokerage that
Fannie failure \ of a


In [13]:
class DownStreamDataset:
    def __init__(self, dataset, tokenizer, max_length):
        self.dataset = dataset['Description']
        self.labels = dataset['Class Index']
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        tokenized_sentence = self.tokenizer(re.sub(r'[^\w\s\']', '',self.dataset[idx].lower()))
        return [START_TOKEN]+tokenized_sentence+[END_TOKEN],self.labels[idx]

         
Down_data = DownStreamDataset(df,nltk.word_tokenize, 100)
print(Down_data[0])
def collate(batch):
    inputs = []
    targets = []
    for i in range(len(batch)):
        inputs.append(torch.tensor([vocab[word] if word in vocab else vocab[UNKNOWN_TOKEN] for word in batch[i][0]]))
        targets.append(batch[i][1])
    targets = torch.tensor(targets)
    return pad_sequence(inputs, batch_first=True, padding_value=vocab[PAD_TOKEN]),targets

(['<s>', 'unions', 'representing', 'workers', 'at', 'turner', 'newall', 'say', 'they', 'are', "'disappointed", "'", 'after', 'talks', 'with', 'stricken', 'parent', 'firm', 'federal', 'mogul', '</s>'], 3)


In [14]:
Down_data_loader = DataLoader(Down_data, batch_size=32, shuffle=True, collate_fn=collate)
for i in range(1):
    for inputs, targets in Down_data_loader:
        print(inputs.shape)
        print(targets.shape)
        break

torch.Size([32, 56])
torch.Size([32])


In [49]:
class DownStreamClassifier(torch.nn.Module):
  def __init__(self,num_label,elmo,hidden_dim,embed_dim,num_layers):
    super().__init__()

    
    self.lstm = torch.nn.LSTM(embed_dim,hidden_dim,num_layers,batch_first=True,bidirectional=True)
    self.outLayer = torch.nn.Linear(2*hidden_dim,num_label)
    self.num_classes = num_label
    self.elmo = elmo
    self.embed_func = torch.nn.Linear(6*hidden_dim,embed_dim)
    self.elmo.to(device)

  def forward(self, sentence):
    self.elmo.eval()
    x,of1,of2,or1,or2 = self.elmo.forward(sentence.to(device))
    first_layer = torch.cat((of1,or1),axis=2)
    second_layer = torch.cat((of2,or2),axis=2)
    x = torch.cat((x,x),axis=2)
    embedding = self.embed_func(torch.cat((first_layer,second_layer,x),axis=2))
    lstm_out,_ = self.lstm(embedding)
    lstm_out = lstm_out[:,-1,:]
    output = self.outLayer(lstm_out)
    return torch.nn.functional.softmax(output,dim=1)



In [51]:
print(len(Down_data_loader))
DownStreamModel = DownStreamClassifier(4,LanguageModel,128,512*2,1)
for name, param in DownStreamModel.named_parameters():
    if name.split('.')[0] == 'elmo':
        param.requires_grad = False
for name, param in DownStreamModel.named_parameters():
    if param.requires_grad:
        print(name,param.shape)



238
lambdas torch.Size([3])
lstm.weight_ih_l0 torch.Size([512, 1024])
lstm.weight_hh_l0 torch.Size([512, 128])
lstm.bias_ih_l0 torch.Size([512])
lstm.bias_hh_l0 torch.Size([512])
lstm.weight_ih_l0_reverse torch.Size([512, 1024])
lstm.weight_hh_l0_reverse torch.Size([512, 128])
lstm.bias_ih_l0_reverse torch.Size([512])
lstm.bias_hh_l0_reverse torch.Size([512])
outLayer.weight torch.Size([4, 256])
outLayer.bias torch.Size([4])


In [54]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix



# lstm_test_dataloader = DataLoader(lstm_val_dataset, batch_size=3,collate_fn=lstm_data.collate)
DownStreamModel.float()
DownStreamModel.to(device)
loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(DownStreamModel.parameters(),lr = 0.01)

for epoch_num in range(10):
  # Put model in training mode
  DownStreamModel.train()
  for batch_num, (sentences, tags) in enumerate(Down_data_loader):
    sentences = sentences.to(device)
    tags = tags.to(device)
    
    pred = DownStreamModel(sentences)
    loss = loss_fn(pred,tags-1)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
 
    print(f"{batch_num=}, {loss.item()=}")
      



batch_num=0, loss.item()=-0.24496591091156006
batch_num=1, loss.item()=-0.24247388541698456
batch_num=2, loss.item()=-0.24708682298660278
batch_num=3, loss.item()=-0.25897201895713806
batch_num=4, loss.item()=-0.253031849861145
batch_num=5, loss.item()=-0.2507251501083374
batch_num=6, loss.item()=-0.2511810064315796
batch_num=7, loss.item()=-0.2567189931869507
batch_num=8, loss.item()=-0.24696293473243713
batch_num=9, loss.item()=-0.2605348229408264
batch_num=10, loss.item()=-0.24969173967838287


KeyboardInterrupt: 

In [25]:
csv_file_test = 'iNLP-A2/ANLP-2/test.csv'
tf = pd.read_csv(csv_file_test)
tf = tf.head(100)
test_data = DownStreamDataset(tf,nltk.word_tokenize, 100)
test_data_loader = DataLoader(test_data, batch_size=64, shuffle=True,collate_fn=collate)

DownStreamModel.eval()

true_labels = []
predicted_labels = []

# Iterate over test set
for sentences, labels in test_data_loader:
    # Forward pass
    output = DownStreamModel(sentences)
    labels = labels-1

    # Get predicted labels
    
    _, predicted = torch.max(output, 1)
    print(predicted,labels)
    
    # Append true and predicted labels
    true_labels.extend(labels.view(-1).tolist())
    predicted_labels.extend(predicted.tolist())

# Compute evaluation metrics
print(true_labels,predicted)
accuracy = accuracy_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels,average="weighted")
f1 = f1_score(true_labels, predicted_labels,average="weighted")
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

tensor([2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]) tensor([0, 3, 0, 0, 3, 3, 3, 2, 1, 2, 1, 3, 3, 0, 1, 1, 0, 0, 3, 0, 3, 2, 0, 2,
        0, 3, 1, 0, 2, 1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 1, 2, 3, 3, 1, 3, 2, 1, 3,
        1, 0, 1, 2, 3, 3, 3, 1, 3, 0, 2, 0, 0, 1, 1, 3])
tensor([2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2]) tensor([3, 0, 3, 3, 0, 3, 1, 1, 3, 0, 2, 3, 3, 3, 0, 3, 3, 0, 0, 3, 3, 1, 3, 3,
        3, 2, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2])
[0, 3, 0, 0, 3, 3, 3, 2, 1, 2, 1, 3, 3, 0, 1, 1, 0, 0, 3, 0, 3, 2, 0, 2, 0, 3, 1, 0, 2, 1, 1, 3, 3, 3, 3, 0, 0, 0, 0, 1, 2, 3, 3, 1, 3, 2, 1, 3, 1, 0, 1, 2, 3, 3, 3, 1, 3, 0, 2, 0, 0, 1, 1, 3, 3, 0, 3, 3, 0, 3, 1, 1, 3, 0, 2, 3, 3, 3, 0, 3, 3, 0, 0, 3, 3, 1, 3, 3, 3, 2, 0, 0, 1, 1, 1, 0, 0, 0, 0, 2] tensor([2, 2, 2